<a href="https://colab.research.google.com/github/Gustavo-Ros/Simulacion-1/blob/main/Ejercicio_CossBu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EJERCICIO 5.7

Un vendedor de revistas compra mensualmente una revista el día primero de cada mes. El costo de cada ejemplar es de $1.50. La demanda de esta revista en los primeros 10 días del mes sigue la siguiente distribución de probabilidad:

\begin{matrix}
Demanda & 5 & 6 & 7 & 8 & 9 & 10 & 11 \\
Probabilidad & 0.05 & 0.05 & 0.10 & 0.15 & 0.25 & 0.25 & 0.15
\end{matrix}


Al final del décimo día, el vendedor puede regresar cualquier cantidad al proveedor, quien se las pagará a \$0.60  el ejemplar, o comprar más a \\$1.20 el ejemlar. La demadna en los siguientes 20 días está dada por las siguiente distribución de probabilidad:

\begin{matrix}
Demanda & 4 & 5 & 6 & 7 & 8  \\
Probabilidad & 0.15 & 0.20 & 0.30 & 0.20 & 0.15
\end{matrix}

Al final del mes, el vendedor puede regresar al proveedor las revistas que le sobren, las cuales se le pagarán a \\$0.60 el ejemplar. Finalmente, se asume que después de un mes ya no existe demanda por parte delpúblico, puesto que para ese entonces ya habría aparecido el nuevo número de la revista. Si el precio al público es de \$2 por ejemplar, determine la política óptima de compra.

#CÓDIGO

In [238]:
#Se cargan las líbrerias
import random as rnd
import numpy as np
from tabulate import tabulate
import matplotlib.pyplot as plt
import pandas as pd
from scipy.optimize import minimize_scalar

In [239]:
def demanda_1():                          #Esta función calcula y devuelve la demanda del 1er periodo
  prob=[0,0.05,0.1,0.2,0.35,0.6,0.85,1]
  u=rnd.random()
  for i in range(len(prob)):
      if prob[i-1] <= u <= prob[i]:       #Ubica el valor de u en inervalos
          x=i+4                           #Asigna valor de demanda
          break
  return x

In [240]:
def demanda_2():                          #Esta función calcula y devuelve la demanda del 2do periodo
  prob=[0,0.15,0.35,0.65,0.85,1]
  u=rnd.random()
  for i in range(len(prob)):
      if prob[i-1] <= u <= prob[i]:       #Ubica el valor de u en inervalos
          x=i+3                           #Asigna valor de demanda
          break
  return x

In [241]:
def ganancias(ganancia,demanda,revistas,gastos,precio_venta):    #  Calcula las ganancias del periodo en curso

  if demanda > revistas:            #Este condicional permite vender solo las revistas disponibles
    ventas=revistas*precio_venta
  else:
    ventas=demanda*precio_venta

  ganancia+=ventas-gastos           #Calcula las ganancias de vender las revistas
  return ganancia,ventas

In [242]:
def PERIODO_1(compra1,costo_revistas1,precio_venta):  #Calcula los resultados del primer periodo
  tabla=[]
  gastos=compra1*costo_revistas1        # Calcula los gastos
  ganancia_total=0
  demanda1=demanda_1()                  # Calcula la demanda en el primer periodo

  if compra1-demanda1 <= 0:             # Decide si hay inventario disponible o no
    inventario=0
  else:
    inventario=compra1-demanda1

  # Calcula las ventas y ganancias de las ventas en el primer periodo
  ganancia_total,venta=ganancias(ganancia_total,demanda1,compra1,gastos,precio_venta)
  # Guarda resultados en una tabla
  tabla.append(["Periodo 1",f"{compra1}",f"{compra1}",f"{demanda1}",f"{gastos}",f"{venta}",f"{inventario}",f"{ganancia_total}"])
  return inventario,ganancia_total,tabla

In [244]:
 # Calcula los resultados del segundo periodo
 def PERIODO_2(inventario,ganancia_total,compra2,costo_revistas2,precio_venta,tabla,sobrantes):

  demanda2=demanda_2()        # Calcula la demanda en el segundo periodo

  if inventario < 4:          # Decide si hay revistas sobrantes para vender
    sobrantes=0
  elif inventario >= 8:
    sobrantes=inventario-8
    compra2=0

  gastos2=compra2*costo_revistas2     # Calcula los gastos de la segunda compra de revistas

  if inventario+compra2-demanda2-sobrantes <= 0:
    inventario=0

  if inventario+compra2-sobrantes-demanda2<=0:    # Decide si hay inventario disponible o no
    inv_final=0
  else:
    inv_final=inventario+compra2-sobrantes-demanda2

  # Calcula las ventas y ganancias de las ventas en el primer periodo
  ganancia_total,venta=ganancias(ganancia_total,demanda2,inventario+compra2-sobrantes,gastos2,precio_venta)
  # Guarda resultados en una tabla
  tabla.append(["Periodo 2",f"{compra2}",f"{inventario+compra2-sobrantes}",f"{demanda2}",f"{gastos2}",f"{venta}",f"{inv_final}",f"{ganancia_total}"])

  return inventario,ganancia_total,tabla,inv_final

## RESULTADOS DE 1 ITERACIÓN

In [234]:
 def ejercicio_revistas(compra1,compra2):

  #------------------------------ PARÁMETROS INICIALES ---------------------------
  costo_revistas1=1.5
  sobrantes_final=0.6
  costo_revistas2=1.2
  precio_sobrantes=0.9
  precio_venta=2
  sobrantes=0

  #-------------------------------  PERIODO 1 : Primeros 10 días------------------
  inventario,ganancia_total,tabla=PERIODO_1(compra1,costo_revistas1,precio_venta)

  #-------------------------------  PERIODO 2 : Siguientes 20 días----------------
  inventario,ganancia_total,tabla,inv_final=PERIODO_2(inventario, ganancia_total,compra2,costo_revistas2,precio_venta,tabla,sobrantes)

  headers=["","Revistas compradas","Revistas totales","Demanda","Costo revistas","Venta de revistas","Inventario final","Ganancia"]
  print(tabulate(tabla, headers=headers, tablefmt="fancy_grid",showindex=False))

  ganancia_total+=inv_final*sobrantes_final

  #-------------------------------  RESULTADOS FIN DE MES ------------------------

  print("Al final del periodo 1 se vendieron ", sobrantes, "revistas")

  if ganancia_total<0:
    print('La pérdida al final del mes es: ', ganancia_total)
  else:
    print('La ganancial total al final del mes es: $', ganancia_total)
  return ganancia_total

In [232]:
compra1=rnd.randint(1,19)
compra2=rnd.randint(4,8)
ganancias_mes=ejercicio_revistas(compra1,compra2)

╒═══════════╤══════════════════════╤════════════════════╤═══════════╤══════════════════╤═════════════════════╤════════════════════╤════════════╕
│           │   Revistas compradas │   Revistas totales │   Demanda │   Costo revistas │   Venta de revistas │   Inventario final │   Ganancia │
╞═══════════╪══════════════════════╪════════════════════╪═══════════╪══════════════════╪═════════════════════╪════════════════════╪════════════╡
│ Periodo 1 │                   17 │                 17 │        11 │             25.5 │                  22 │                  6 │       -3.5 │
├───────────┼──────────────────────┼────────────────────┼───────────┼──────────────────┼─────────────────────┼────────────────────┼────────────┤
│ Periodo 2 │                    8 │                 14 │         6 │              9.6 │                  12 │                  8 │       -1.1 │
╘═══════════╧══════════════════════╧════════════════════╧═══════════╧══════════════════╧═════════════════════╧════════════════════

## OPTIMIZACIÓN DE RESULTADOS

In [245]:
 def opti_ejercicio_revistas(compra1,compra2):

  #------------------------------ PARÁMETROS INICIALES ---------------------------
  costo_revistas1=1.5
  sobrantes_final=0.6
  costo_revistas2=1.2
  precio_sobrantes=0.9
  precio_venta=2
  sobrantes=0

  #-------------------------------  PERIODO 1 : Primeros 10 días------------------
  inventario,ganancia_total,tabla=PERIODO_1(compra1,costo_revistas1,precio_venta)

  #-------------------------------  PERIODO 2 : Siguientes 20 días----------------
  inventario,ganancia_total,tabla,inv_final=PERIODO_2(inventario, ganancia_total,compra2,costo_revistas2,precio_venta,tabla,sobrantes)

  ganancia_total+=inv_final*sobrantes_final

  return ganancia_total

In [246]:
# Este bloque de código encuentra las compras óptimas de revistas  en los periodos 1 y 2
mejor_ganancia = float('-inf')
mejor_compra1 = None
mejor_compra2 = None
iteraciones = 1000  # Número de iteraciones a realizar

for _ in range(iteraciones):        # Realiza 1000 iteraciones para explorar resultados
    compra1 = rnd.randint(1, 19)
    compra2 = rnd.randint(4, 8)
    ganancias_mes = opti_ejercicio_revistas(compra1, compra2)

    if ganancias_mes > mejor_ganancia:  # Actualiza la mejor ganancia y las compras óptimas
        mejor_ganancia = ganancias_mes
        mejor_compra1 = compra1
        mejor_compra2 = compra2

# Imprime resultados

print(f"Máxima ganancia: ${mejor_ganancia}")
print(f"Compra 1 óptima: {mejor_compra1}")
print(f"Compra 2 óptima: {mejor_compra2}")


Máxima ganancia: $11.9
Compra 1 óptima: 11
Compra 2 óptima: 8
